# Graph-tool tutorial

The following tutorial demonstrates working with graphs using the [graph-tool python module](https://graph-tool.skewed.de/). In the process, you will learn how to:

* create a simple graphs,
* perform basic network analysis,
* visualize graphs and their properties.

For additional background on the parameters for the drawing routines, see the [graph-tool tutorial](https://stevenbergner.github.io/Teaching/cmpt767/graph-tutorial)

In [8]:
import pandas as pd
import numpy as np
import graph_tools as gt
%matplotlib inline
# print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

# Show datasets in collection

In [6]:
with pd.option_context('display.max_colwidth', -1):
    display(pd.DataFrame.from_records(gt.collection.descriptions,
                                      index=['description']).transpose())

AttributeError: module 'graph_tools' has no attribute 'collection'

In [ ]:
g = gt.collection.data['karate']
g

In [ ]:
list(g.vp.keys())

In [ ]:
gt.graph_draw(g, pos=g.vp['pos'])

In [ ]:
X_knows = {
    'Mary': ['Peter', 'Albert', 'DavidF', 'Peter'],
    'Judy': ['Bob', 'Alan'],
    'Peter': ['Mary', 'DavidF', 'Jon'],
    'DavidF': ['Albert', 'Joseph', 'Peter', 'Mary'],
    'Jon': ['Peter', 'Joseph', 'DavidE'],
    'DavidE': ['Jon', 'Joseph', 'Albert'],
    'Joseph': ['DavidE', 'Jon', 'DavidF'],
    'Bob': ['Judy', 'Alan'],
    'Alan': ['Bob', 'Mary', 'Judy'],
    'Albert': ['DavidF', 'Mary', 'DavidE'],
}

In [ ]:
g = gt.Graph(directed=True)

In [ ]:
v_name = g.add_edge_list(((n,k) for n in X_knows for k in X_knows[n]),
                         hashed=True)
g.vp['name'] = v_name

In [ ]:
pos = gt.arf_layout(g)
#pos = gt.planar_layout(g)
#pos = gt.fruchterman_reingold_layout(ug)

In [ ]:
gt.graph_draw(
    ug,
    pos=pos,
    vertex_fill_color="#dff0ff",
    vertex_text=v_name,
    vertex_size=10,
)

In [ ]:
#from functools import *
#X_names = reduce(lambda a,b: set(a).union(b), X_knows.values()).union(X_knows.keys())

In [ ]:
gt.graphviz_draw(
    g,
    output="output.svg",
    output_format="svg"
)

![](output.svg)

# Create a graph using Python iterations

Below is a slow and tedious version of what can be done with a single call to `add_edge_list(...)` on a `Graph`.

In [ ]:
# Create edge tuples and list of unique names
X_edges = list((n,k) for n in X_knows for k in X_knows[n])
from functools import reduce
X_names = reduce(lambda a,b: set(a).union(b),
                 (X_knows[n] for n in X_knows)
                ).union(X_knows.keys())
X_names = list(X_names)

In [ ]:
# Construct a 'StringIndexer' to convert strings to integers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
lem = le.fit(list(X_names))
X_edges = list(map(lem.transform, X_edges))

In [ ]:
# Create Graph object and add a string property for names
g2 = gt.Graph()
v_name = g2.new_vertex_property('string')
g2.vertex_properties['name'] = v_name

In [ ]:
for vn in lem.classes_:
    v = g2.add_vertex()
    v_name[v] = vn

In [ ]:
for f,t in X_edges:
    g2.add_edge(f,t)

In [ ]:
g3 = gt.GraphView(g2, directed=False)
gt.remove_parallel_edges(g3)
gt.graph_draw(g3)

# Another graph example
From graph-tools [search module](https://graph-tool.skewed.de/static/doc/search_module.html)

In [ ]:
# download the xml source, if missing
#!wget https://stevenbergner.github.io/Teaching/cmpt767/search_example.xml

g = gt.load_graph("search_example.xml")
name = g.vp["name"]
weight = g.ep["weight"]
pos = g.vp["pos"]
gt.graph_draw(g, pos, vertex_text=name, vertex_font_size=12, vertex_shape="double_circle",
              vertex_fill_color="#729fcf", vertex_pen_width=1,
              edge_pen_width=weight)
